<a href="https://colab.research.google.com/github/saridsa1/med2vec/blob/master/Med2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/saridsa1/med2vec.git

Cloning into 'med2vec'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 68 (delta 0), reused 3 (delta 0), pack-reused 64
Unpacking objects: 100% (68/68), done.


In [0]:
import sys
import cPickle as pickle
from datetime import datetime

def convert_to_icd9(dxStr):
	if dxStr.startswith('E'):
		if len(dxStr) > 4: return dxStr[:4] + '.' + dxStr[4:]
		else: return dxStr
	else:
		if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
		else: return dxStr
	
def convert_to_3digit_icd9(dxStr):
	if dxStr.startswith('E'):
		if len(dxStr) > 4: return dxStr[:4]
		else: return dxStr
	else:
		if len(dxStr) > 3: return dxStr[:3]
		else: return dxStr

admissionFile = '/content/med2vec/admissions.csv' #/content/med2vec/admissions.csv
diagnosisFile = '/content/med2vec/diagnosis.csv' #/content/med2vec/diagnosis.csv
outFile = '/content/med2vecinput'

print 'Building pid-admission mapping, admission-date mapping'
pidAdmMap = {}
admDateMap = {}
infd = open(admissionFile, 'r')
infd.readline()
for line in infd:
  tokens = line.strip().split(',')
  pid = int(tokens[1])
  admId = int(tokens[2])
  admTime = datetime.strptime(tokens[3], '%Y-%m-%d %H:%M:%S')
  admDateMap[admId] = admTime
  if pid in pidAdmMap: pidAdmMap[pid].append(admId)
  else: pidAdmMap[pid] = [admId]
infd.close()

print 'Building admission-dxList mapping'
admDxMap = {}
admDxMap_3digit = {}
infd = open(diagnosisFile, 'r')
infd.readline()
for line in infd:
  tokens = line.strip().split(',')
  admId = int(tokens[2])
  dxStr = 'D_' + convert_to_icd9(tokens[3][1:-1]) ############## Uncomment this line and comment the line below, if you want to use the entire ICD9 digits.
  dxStr_3digit = 'D_' + convert_to_3digit_icd9(tokens[3][1:-1])

  if admId in admDxMap: 
    admDxMap[admId].append(dxStr)
  else: 
    admDxMap[admId] = [dxStr]

  if admId in admDxMap_3digit: 
    admDxMap_3digit[admId].append(dxStr_3digit)
  else: 
    admDxMap_3digit[admId] = [dxStr_3digit]
infd.close()

print 'Building pid-sortedVisits mapping'
pidSeqMap = {}
pidSeqMap_3digit = {}
for pid, admIdList in pidAdmMap.iteritems():
  if len(admIdList) < 2: continue

  sortedList = sorted([(admDateMap[admId], admDxMap[admId]) for admId in admIdList])
  pidSeqMap[pid] = sortedList

  sortedList_3digit = sorted([(admDateMap[admId], admDxMap_3digit[admId]) for admId in admIdList])
  pidSeqMap_3digit[pid] = sortedList_3digit

print 'Building pids, dates, strSeqs'
pids = []
dates = []
seqs = []
for pid, visits in pidSeqMap.iteritems():
  pids.append(pid)
  seq = []
  date = []
  for visit in visits:
    date.append(visit[0])
    seq.append(visit[1])
  dates.append(date)
  seqs.append(seq)

print 'Building pids, dates, strSeqs for 3digit ICD9 code'
seqs_3digit = []
for pid, visits in pidSeqMap_3digit.iteritems():
  seq = []
  for visit in visits:
    seq.append(visit[1])
  seqs_3digit.append(seq)

print 'Converting strSeqs to intSeqs, and making types'
types = {}
newSeqs = []
for patient in seqs:
  newPatient = []
  for visit in patient:
    newVisit = []
    for code in visit:
      if code in types:
        newVisit.append(types[code])
      else:
        types[code] = len(types)
        newVisit.append(types[code])
    newPatient.append(newVisit)
  newSeqs.append(newPatient)

print 'Converting strSeqs to intSeqs, and making types for 3digit ICD9 code'
types_3digit = {}
newSeqs_3digit = []
for patient in seqs_3digit:
  newPatient = []
  for visit in patient:
    newVisit = []
    for code in set(visit):
      if code in types_3digit:
        newVisit.append(types_3digit[code])
      else:
        types_3digit[code] = len(types_3digit)
        newVisit.append(types_3digit[code])
    newPatient.append(newVisit)
  newSeqs_3digit.append(newPatient)

print 'Re-formatting to Med2Vec dataset'
seqs = []
for patient in newSeqs:
  seqs.extend(patient)
  seqs.append([-1])
seqs = seqs[:-1]

seqs_3digit = []
for patient in newSeqs_3digit:
  seqs_3digit.extend(patient)
  seqs_3digit.append([-1])
seqs_3digit = seqs_3digit[:-1]

pickle.dump(seqs, open(outFile+'.seqs', 'wb'), -1)
pickle.dump(types, open(outFile+'.types', 'wb'), -1)
pickle.dump(seqs_3digit, open(outFile+'.3digitICD9.seqs', 'wb'), -1)
pickle.dump(types_3digit, open(outFile+'.3digitICD9.types', 'wb'), -1)

Building pid-admission mapping, admission-date mapping
Building admission-dxList mapping
Building pid-sortedVisits mapping
Building pids, dates, strSeqs
Building pids, dates, strSeqs for 3digit ICD9 code
Converting strSeqs to intSeqs, and making types
Converting strSeqs to intSeqs, and making types for 3digit ICD9 code
Re-formatting to Med2Vec dataset


In [0]:
import sys
sys.path.append('/content/med2vec')

In [0]:
!python /content/med2vec/med2vec.py '/content/med2vecinput.3digitICD9.seqs' 942 '/content/med2vecoutput'

initializing parameters
building models
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
loading data
training start
epoch:0, mean_cost:68.983670
epoch:1, mean_cost:39.997908
epoch:2, mean_cost:38.035020
epoch:3, mean_cost:36.978985
epoch:4, mean_cost:36.064517
epoch:5, mean_cost:35.249457
epoch:6, mean_cost:34.043896
epoch:7, mean_cost:35.143960
epoch:8, mean_cost:36.392260
epoch:9, mean_cost:31.220896
